<h1>강화학습</h1>

In [ ]:
# gym 라이브러리 설치
# 강화학습 연구와 교육을 위해 각종 환경과 API를 제공
!pip install gym==0.23

In [ ]:
# 라이브러리
import gym
import random
import numpy as np
import matplotlib.pyplot as plt
# 경고 메시지 출력하지 않기
import warnings
warnings.filterwarnings('ignore')

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


# Fronzen Lake + Q-Learning

환경 설정

In [ ]:
# 강의에서 다룬 Fronzen Lake 예제
# 상태: 구멍(H)-출발(S)-얼음(F)-얼음(F)-목적지(G) 의 구조
# 행동: 0(LEFT), 1(DOWN), 2(RIGHT), 3(UP)
#       여기서는 2개만 허용: 0(LEFT), 2(RIGHT)
# 보상: G은 +1, 나머지는 0
env = gym.make('FrozenLake-v1',desc=['HSFFG'],is_slippery=False)

가치함수와 정책함수

In [ ]:
# 가치함수: Q테이블, 0으로 초기화
Q = np.zeros([env.observation_space.n, env.action_space.n])
Q

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [ ]:
# 정책 함수, e는 e-greedy 변수
# 4의 행동에 대한 Q값(L,D,R,U)을 받아 행동을 결정
# 이 중에서 L/R만 사용
def policy(qs,e=0):
  if e > random.random():
    return random.choice([0,2])
  else:
    if qs[0] > qs[2]: return 0
    elif qs[0] < qs[2]: return 2
    else: return random.choice([0,2])

모델 훈련

In [ ]:
num_episodes = 100   # 필요하다면 더 늘려서 수행
discount = 0.9
Q = Q*0.0  # Q함수 초기화
for i in range(num_episodes):
  state = env.reset()   # 초기 상태로 리셋
  done = False
  while not done:
    action = policy(Q[state, :],e=0.1) # 행동 결정
    new, reward, done, _ = env.step(action) # 행동에 따른 보상과 새로운 상태
    Q[state,action] = reward + discount*np.max(Q[new, :]) # Q함수 업데이트
    state = new  # 상태 업데이트

In [ ]:
Q[1:-1,[0,2]]  # 최종 Q함수

array([[0.   , 0.81 ],
       [0.729, 0.9  ],
       [0.81 , 1.   ]])

# 연습문제

1. 위의 예제에서 다음을 변경하며 수행하시오.<br>
(1) policy 함수에서 e를 0.2으로 변경하였을 때, 최종적인 Q함수는 어떻게 주어지는가? [sH,s0,s1,s2,sD]와 같이 상태를 표시할 때, Q(s2,L)의 값은 얼마인가? 왜 그런지 설명하시오.<br>
(2) discount가 0.7일 때, policy 함수에서 e를 0.1로 두고 충분히 학습시키면 최종적인 Q함수는 어떻게 주어지는가?

2. 위의 예제에서 아래와 같이 환경을 생성하고 Q학습을 수행하시오. 이 경우는 2차원 공간이기 때문에 4개의 행동이 모두 허용되어야 한다. policy 함수 및 다른 코드를 적절히 변경해가며 수행하시오. 최종적인 Q함수가 적절한지 확인하시오.

In [ ]:
env = gym.make('FrozenLake-v1',desc=["SFFF","FHFH","FFFH","HFFG"],is_slippery=False)